In [ ]:
import numpy as np
import cvxpy as cp

In [ ]:
#HW5
x = np.array([2,3,5,6,8])
y= np.array([12,11,19,21,34])
sxy= sum([(x[i]-np.mean(x))*(y[i]) for i in range(5)])
sxx= sum([(x[i]-np.mean(x))**2 for i in range(5)])
beta= sxy/sxx
alpha = np.mean(y)-beta*np.mean(x)
print(f'The optimal alpha={alpha},beta={beta }')


The optimal alpha=1.6315789473684141,beta=3.7017543859649136


In [ ]:
np.mean(y)

19.4

In [ ]:
from scipy.stats import norm


3.121389149359862

## Data for Problem 1
(a) find choice of locations that guarantees that there is at least one in reasonable distance of each dorm

Let $x_i$ denote whether to open or not at location i
$$
\begin{array}{rl} 
min & \sum_{i=1}^Lc_i*x_i \\
\mbox{s.t}& \sum_iS_j *x_i >=1 & \mbox{for very j}\in \{1,..D\} \\
& x_i \in \{0,1\}
\end{array}
$$

In [ ]:
# set the seed for the random number generator
# DO NOT CHANGE THE SEED!
np.random.seed(2) 


### Data for the pizza problem

L = 20
c = np.ceil(10*np.random.rand(L)) #cost 
n = 2 # This is only used in Problem 1 part (b)

D = 25
p = np.ceil(100*np.random.rand(D))
# A location i is acceptable for dorm j is S[j,i] = 1
S = np.random.randint(2,size=(D,L))
x = cp.Variable(L,boolean=True) # decision variable 

## (a)
obj = cp.Minimize(c@x) 
constraint = [S@x >=1]
bin_prob = cp.Problem(obj,constraint) 
bin_prob.solve()
if (bin_prob.status == "optimal"):
    print('x = ' + str(x.value))



x = [0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]


In [ ]:
a =x.value
index_ = np.where(a==1)
for i in list(index_):
    print(f'location {i+1} will open')


location [ 2 10 15 20] will open


##(b)
Whether dorm j will get covered depends on if there is one location i deliverable (S[j,i]) and whether that location i is open. 

$$
\begin{array}{rl} 
max & \sum_{i=j}^D y_j*p_j \\
\mbox{s.t}& \sum_ix_i <=n &  \\
& y_j <= \sum_i S_{ji}*x_i, \mbox{for every dorm j} \\
& x_i,y_i \in \{0,1\}
\end{array}
$$

In [ ]:
#(b)
y= cp.Variable(D,boolean=True) # whether dorm j will get covered
obj=cp.Maximize(y@p)
constraint = [sum(x) <=n,y<=S*x]

bin_prob = cp.Problem(obj,constraint) 
bin_prob.solve()
if (bin_prob.status == "optimal"):
    print('Pizza location that will open x = ' + str(x.value))
    print('Maximum students served is ', bin_prob.value)
    print('Dorms that will get corverd y=',y.value)
print('Location 17,19 will optimize the coverage')


Pizza location that will open x = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
Maximum students served is  1039.0
Dorms that will get corverd y= [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
Location 17,19 will optimize the coverage


In [ ]:
for i in range(L):
    for j in range(D):
        constraint +=[x[i]+y[j]-1<=S[j,i], x[i]>=S[j,i],y[j]>=S[j,i]]

## Data for Problem 2
$$
\begin{array}{rl} 
min & \sum_{i=1}^8 |x_i-vi|\\
\mbox{s.t}& \sum_i k_i <=3  \\
\mbox{for all i} \\
& u_i >= x_i-v_i \\
& u_i >= - (x_i-v_i) \\
& w_i >= b_i-x_i \\
& w_i >=-(b_i-x_i) \\ 
& k_i*|b_i-v_i| >= w_i \\
& k_i \in \{0,1\} \\
&x_i >=0
\end{array}
$$

In [ ]:
## Data for Problem 2
b = np.array([0.12,0.15,0.13,0.10,0.20,0.10,0.12,0.08])
v = np.array([0.02,0.05,0.25,0.06,0.18,0.10,0.22,0.12])
K = 3
x = cp.Variable(8)
k =cp.Variable(8, boolean=True) # if k=0, not rebalace
u=cp.Variable(8)
w=cp.Variable(8)
obj = cp.Minimize(sum(u))
constraint =[sum(k)<=3,sum(x)==1]
for i in range(8):
    constraint += [
              u[i]>= x[i]-v[i],
              u[i]>=-(x[i]-v[i]),
              w[i]>=b[i]-x[i],
              w[i]>= -b[i]+x[i],
              k[i]*abs(b[i]-v[i])>=w[i],
              
            
              x[i]>=0
              ]
bin_prob = cp.Problem(obj,constraint) 
bin_prob.solve()
if (bin_prob.status == "optimal"):
    print('rebalanced portfolio is  = ' + str(x.value))
    print("total error is",obj.value)

rebalanced portfolio is  = [0.1  0.05 0.25 0.1  0.2  0.1  0.12 0.08]
total error is 0.2800000000000001


## Data for Problem 3

In [ ]:
# set the seed for the random number generator
# DO NOT CHANGE THE SEED!
np.random.seed(15) 

n = 10

v = np.ceil(10*np.random.rand(n))
b = np.ceil(10*(np.random.rand(n,n)-np.random.rand(n,n)))
w = np.ceil(10*np.random.rand(n))
W = np.ceil(0.75*sum(w))

x = cp.Variable(n,boolean=True)
z = cp.Variable((n,n), boolean=True)
obj = sum([ v[i]*x[i] for i in range(n)])
for i in range(n):
    for j in range(n):
        obj +=b[i,j]*z[i,j]
obj =cp.Maximize( obj )
constraint = [sum([w[i]*x[i] for i in range(n)]) <=W]
for i in range(n):
    for j in range(n):
        constraint += [
              z[i,j] >= x[i]+x[j]-1,
              z[i,j]<=x[i],
              z[i,j]<=x[j]


]

prob = cp.Problem(obj,constraint) 
prob.solve()
if (prob.status == "optimal"):
    print('x = ' + str(x.value))
    print(obj.value)


x = [1. 0. 0. 1. 1. 1. 0. 1. 1. 0.]
76.0


array([[1., 0., 1., 1., 1., 1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 1.]])